In [87]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
#input train data from csv
from proj1_helpers import *
DATA_TRAIN_PATH = 'D:\\Jupyter Notebook\Machine Learning\project1\data\\train.csv'  
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [89]:
#check the shape of y,tX,ids
y.shape,tX.shape,ids.shape

((250000,), (250000, 30), (250000,))

### 1 Feature Engineering

#### 1.1 import some functions and make some settings

In [90]:
def p_x(p_t,phi):
    
    length = len(p_t)
    px = np.zeros(length)
    valid_index = (p_t!=-999)
    
    px[valid_index] = p_t[valid_index]*np.cos(phi[valid_index])
    px[~valid_index] = -999
        
    return px

def p_y(p_t,phi):

    length = len(p_t)
    py = np.zeros(length)
    valid_index = (p_t!=-999)
    
    py[valid_index] = p_t[valid_index]*np.sin(phi[valid_index])
    py[~valid_index] = -999
    
    return py

def p_z(p_t,eta):

    length = len(p_t)
    pz = np.zeros(length)
    valid_index = (p_t!=-999)
    
    pz[valid_index] = p_t[valid_index]*np.sinh(eta[valid_index])
    pz[~valid_index] = -999
    
    return pz

#mass are neglected, E = p
def particle_energy(px,py,pz):
    
    length = len(px)
    energy = np.zeros(length)
    valid_index = (px!=-999)
    
    energy[valid_index] = np.sqrt(px[valid_index]**2+py[valid_index]**2+pz[valid_index]**2)
    energy[~valid_index] = -999
    return energy

def cross_product(px_1,py_1,pz_1,px_2,py_2,pz_2):
    N = len(px_1)
    cp_x = np.zeros(N)
    cp_y = np.zeros(N)
    cp_z = np.zeros(N)
    for t in range(N):
        if(px_1[t]!=-999 and px_2[t]!=-999):
            temp_cross=np.cross(np.array([px_1[t],py_1[t],pz_1[t]]),np.array([px_2[t],py_2[t],pz_2[t]]))
            cp_x[t] = temp_cross[0]
            cp_y[t] = temp_cross[1]
            cp_z[t] = temp_cross[2]
        else:
            cp_x[t] = -999
            cp_y[t] = -999
            cp_z[t] = -999
    return cp_x,cp_y,cp_z

def dot_product(px_1,py_1,pz_1,px_2,py_2,pz_2):
    N = len(px_1)
    dp = np.zeros(N)
    for t in range(N):
        if(px_1[t]!=-999 and px_2[t]!=-999):
            dp[t] = np.inner(np.array([px_1[t],py_1[t],pz_1[t]]),np.array([px_2[t],py_2[t],pz_2[t]]))
        else:
            dp[t] = -999
    return dp

def cosine_similarity(px_1,py_1,pz_1,px_2,py_2,pz_2):
    length = len(px_1)
    
    cp = np.zeros(length)
    
    valid_index = (px_1!=-999)&(px_2!=-999)
    

    cp[valid_index] = dot_product(px_1[valid_index],py_1[valid_index],pz_1[valid_index],px_2[valid_index],py_2[valid_index],pz_2[valid_index])/(np.sqrt(px_1[valid_index]**2+py_1[valid_index]**2+pz_1[valid_index]**2)*np.sqrt(px_2[valid_index]**2+py_2[valid_index]**2+pz_2[valid_index]**2))
    cp[~valid_index] = -999
    
    return cp

def determinant_vector(px_1,py_1,pz_1,px_2,py_2,pz_2,px_3,py_3,pz_3):
    N = len(px_1)
    dv = np.zeros(N)
    
    for t in range(N):
        if((px_1[t]!=-999)and(px_2[t]!=-999)and(px_3[t]!=-999)):
            temp_vector = np.array([[px_1[t],py_1[t],pz_1[t]],[px_2[t],py_2[t],pz_2[t]],[px_3[t],py_3[t],pz_3[t]]])
            dv[t] = np.linalg.det(temp_vector)
        else:
            dv[t] = -999;

    return dv

def sum_p_xyz(px,py,pz):
    
    length = len(px)
    sp=np.zeros(length)
    valid_index = (px!=-999)
    sp[valid_index] = px[valid_index]+py[valid_index]+pz[valid_index]
    sp[~valid_index] = -999
    return sp

#Compute Transverse Mass 
def transverse_mass(a_t,a_phi,b_t,b_phi):
    
    length = len(a_t)
    mass = np.zeros(length)
    valid_index = (a_t!=-999)&(b_t!=-999)
    
    mass[valid_index] = np.sqrt((a_t[valid_index]+b_t[valid_index])**2-(a_t[valid_index]*np.cos(a_phi[valid_index])+b_t[valid_index]*np.cos(b_phi[valid_index]))**2-(a_t[valid_index]*np.sin(a_phi[valid_index])+b_t[valid_index]*np.sin(b_phi[valid_index]))**2)
    
    mass[valid_index] = check_transverse_mass(mass[valid_index])
    mass[~valid_index] = -999
    
    return mass

def check_transverse_mass(mass):
    length = len(mass)
    
    median = np.median(mass[np.isnan(mass)==0])
    for t in range(length):
        if(np.isnan(mass[t])==1):
            mass[t] = median
    return mass

def invariant_mass(a_t,a_eta,a_phi,b_t,b_eta,b_phi):
    length = len(a_t)
    
    mass = np.zeros(length)
    
    valid_index = (a_t!=-999)&(b_t!=-999)
    
    
    a_z = a_t[valid_index]*np.sinh(a_eta[valid_index])
    b_z = b_t[valid_index]*np.sinh(b_eta[valid_index])
    
    a_xyz = np.sqrt(a_t[valid_index]**2+(a_z)**2)
    b_xyz = np.sqrt(b_t[valid_index]**2+(b_z)**2)
    ab_x = a_t[valid_index]*np.cos(a_phi[valid_index])+b_t[valid_index]*np.cos(b_phi[valid_index])
    ab_y =a_t[valid_index]*np.sin(a_phi[valid_index])+b_t[valid_index]*np.sin(b_phi[valid_index])
    ab_z =a_z+b_z
    
    
    mass[valid_index] = np.sqrt((a_xyz+b_xyz)**2-(ab_x)**2-(ab_y)**2-(ab_z)**2)
    mass[~valid_index] = -999
    
    return mass

def phi_minus_phi(a_phi,b_phi):
    
    length = len(a_phi)
    result = np.zeros(length)
    
    valid_index = (a_phi!=-999)&(b_phi!=-999)
    
    result[valid_index] = a_phi[valid_index]-b_phi[valid_index]
    result[~valid_index] = -999
    
    return result

In [91]:
def build_poly(tX,degree):
    
    N=tX.shape[0]
    dim = 30
    
    
    for d in range(dim):
        phi = np.zeros((N,degree+1))
        valid_index = (tX[:,d]!=-999)
        for t in range(0,degree+1):
            phi[valid_index,t] = np.power(tX[valid_index,d],t)
            phi[~valid_index,t] = -999
        tX = np.concatenate((tX,phi),axis=1)
    return tX

In [92]:
def build_sqrt(tX):
    N = tX.shape[0]
    dim = 30
    phi = np.zeros((N,30))
    
    
    for d in range(dim):
        valid_index = ((tX[:,d]>=0)&(tX[:,d]!=-999))
        phi[valid_index,d] = np.sqrt(tX[valid_index,d])
        phi[~valid_index,d] = 0
    
    tX = np.concatenate((tX,phi),axis=1)
    
    return tX

In [93]:
def build_log(tX):
    N = tX.shape[0]
    dim = 30
    phi = np.zeros((N,30))
    
    for d in range(dim):
        valid_index = ((tX[:,d]>0)&(tX[:,d]!=-999))
        phi[valid_index,d] = np.log(tX[valid_index,d])
        phi[~valid_index,d] = 0
    
    tX = np.concatenate((tX,phi),axis=1)
    
    return tX


In [94]:
def build_cos_sin(tX):
    N = tX.shape[0]
    dim = 30
    phi_cos = np.zeros((N,30))
    phi_sin = np.zeros((N,30))
    
    for d in range(dim):
        valid_index = (tX[:,d]!=-999)
        phi_cos[valid_index,d] = np.cos(tX[valid_index,d])
        phi_sin[valid_index,d] = np.sin(tX[valid_index,d])
    
    tX = np.concatenate((tX,phi_cos),axis=1)
    tX = np.concatenate((tX,phi_sin),axis=1)
    
    return tX

In [95]:
def build_cross_term(tX):
    N = tX.shape[0]
    dim = 30
    phi = np.zeros((N,int(30*30)))
    t = 0
    
    for d1 in range(dim):
        for d2 in range(dim):
            valid_index = ((tX[:,d1]!=-999)&(tX[:,d2]!=-999))
            phi[valid_index,t] = tX[valid_index,d1]*tX[valid_index,d2]
            t = t+1
    
    tX = np.concatenate((tX,phi),axis=1)
            
    return tX

In [96]:
#import the name of each feature
DER_mass_MMC = tX[:,0]
DER_mass_transverse_met_lep=tX[:,1]
DER_mass_vis = tX[:,2]
DER_pt_h = tX[:,3]
DER_deltaeta_jet_jet = tX[:,4]
DER_mass_jet_jet = tX[:,5]
DER_prodeta_jet_jet = tX[:,6]
DER_deltar_tau_lep = tX[:,7]
DER_pt_tot = tX[:,8]
DER_sum_pt = tX[:,9]
DER_pt_ratio_lep_tau = tX[:,10]
DER_met_phi_centrality = tX[:,11]
DER_lep_eta_centrality = tX[:,12]
PRI_tau_pt = tX[:,13]
PRI_tau_eta = tX[:,14]
PRI_tau_phi = tX[:,15]
PRI_lep_pt = tX[:,16]
PRI_lep_eta =tX[:,17]
PRI_lep_phi = tX[:,18]
PRI_met=tX[:,19]
PRI_met_phi = tX[:,20]
PRI_met_sumet = tX[:,21]
PRI_jet_num=tX[:,22]
PRI_jet_leading_pt=tX[:,23]
PRI_jet_leading_eta = tX[:,24]
PRI_jet_leading_phi = tX[:,25]
PRI_jet_subleading_pt=tX[:,26]
PRI_jet_subleading_eta = tX[:,27]
PRI_jet_subleading_phi = tX[:,28]
PRI_jet_all_pt = tX[:,29]

In [97]:
def data_cleaning(tX):
    
    length = len(tX)
    dim = tX.shape[1]
    delete_index=np.array([],dtype=np.int32)
    

    #for all features
    for d in range(dim):
        if(np.abs(np.std(tX[:,d]))<1e-4):
            delete_index = np.append(delete_index,d)
#         else:
#             mean = np.mean(tX[:,d])
#             std = np.std(tX[:,d])
#             _max = mean+2*std
#             _min = mean-2*std
            
#             tX[:,d][tX[:,d]>_max] = _max
#             tX[:,d][tX[:,d]<_min] = _min
            
#             #calculate again
#             tX[:,d] = (tX[:,d]-np.mean(tX[:,d]))/np.std(tX[:,d])
            
    median = np.median(tX[:,0][tX[:,0]!=-999])
    
    
    #for feature 1
    for t in range(length):
        if(tX[t,0]==-999):
            tX[t,0]==median            
    
    tX=np.delete(tX,delete_index,axis=1)
    
    return tX

#### 1.2 generate some common features

In [98]:
tX=build_poly(tX,6)


In [99]:
tX=build_log(tX)
tX=build_sqrt(tX)
tX=build_cos_sin(tX)

In [100]:
#tX=build_cross_term(tX)

In [101]:
tX.shape

(250000, 360)

In [ ]:
#generate px,py,pz and append to the tX
#tau
p_tau_x = p_x(PRI_tau_pt,PRI_tau_phi)
p_tau_y = p_y(PRI_tau_pt,PRI_tau_phi)
p_tau_z = p_z(PRI_tau_pt,PRI_tau_eta)
#lep
p_lep_x = p_x(PRI_lep_pt,PRI_lep_phi)
p_lep_y = p_y(PRI_lep_pt,PRI_lep_phi)
p_lep_z = p_z(PRI_lep_pt,PRI_lep_eta)
#met
p_met_x = p_x(PRI_met,PRI_met_phi)
p_met_y = p_y(PRI_met,PRI_met_phi)

#append features
tX=np.append(tX,np.array([p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y]).T,axis=1)

#jet leading and subleading x y z

jet_leading_p_x = p_x(PRI_jet_leading_pt,PRI_jet_leading_phi)
jet_leading_p_y = p_y(PRI_jet_leading_pt,PRI_jet_leading_phi)
jet_leading_p_z = p_z(PRI_jet_leading_pt,PRI_jet_leading_eta)

jet_subleading_p_x = p_x(PRI_jet_subleading_pt,PRI_jet_subleading_phi)
jet_subleading_p_y = p_y(PRI_jet_subleading_pt,PRI_jet_subleading_phi)
jet_subleading_p_z = p_z(PRI_jet_subleading_pt,PRI_jet_subleading_eta)

tX=np.append(tX,np.array([jet_leading_p_x,jet_leading_p_y,jet_leading_p_z]).T,axis=1)
tX=np.append(tX,np.array([jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z]).T,axis=1)

In [ ]:
# print(np.count_nonzero(jet_leading_p_x == -999))
# print(np.count_nonzero(jet_leading_p_y == -999))
# print(np.count_nonzero(jet_leading_p_z == -999))
# print(np.count_nonzero(jet_subleading_p_x == -999))
# print(np.count_nonzero(jet_subleading_p_y == -999))
# print(np.count_nonzero(jet_subleading_p_z == -999))

In [ ]:
#generate energy of particle
p_tau_energy = particle_energy(p_tau_x,p_tau_y,p_tau_z)
p_lep_energy = particle_energy(p_lep_x,p_lep_y,p_lep_z)

#append features
tX=np.append(tX,np.array([p_tau_energy,p_lep_energy]).T,axis=1)

#energy of jet leading and subleading
jet_leading_p_energy = particle_energy(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
jet_subleading_p_energy = particle_energy(jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([jet_leading_p_energy,jet_subleading_p_energy]).T,axis=1)

In [ ]:
#transform phi
lep_tau_phi = phi_minus_phi(PRI_lep_phi,PRI_tau_phi)
met_tau_phi = phi_minus_phi(PRI_met_phi,PRI_tau_phi)
jet_tau_phi = phi_minus_phi(PRI_jet_leading_phi,PRI_tau_phi)
jet_sub_tau_phi = phi_minus_phi(PRI_jet_subleading_phi,PRI_tau_phi)

tX=np.append(tX,np.array([lep_tau_phi,met_tau_phi,jet_tau_phi,jet_sub_tau_phi]).T,axis=1)

In [ ]:
N = len(y)
#vector product
#tau-lep

tau_lep_dot = dot_product(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z)
tau_lep_cross_x,tau_lep_cross_y,tau_lep_cross_z = cross_product(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z)
tau_lep_cosine_similarity = cosine_similarity(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z)

#tau-met
tau_met_dot = dot_product(p_tau_x,p_tau_y,p_tau_z,p_met_x,p_met_y,np.zeros(N))
tau_met_cross_x,tau_met_cross_y,tau_met_cross_z = cross_product(p_tau_x,p_tau_y,p_tau_z,p_met_x,p_met_y,np.zeros(N))
tau_met_cosine_similarity =cosine_similarity(p_tau_x,p_tau_y,p_tau_z,p_met_x,p_met_y,np.zeros(N))
#lep-met
lep_met_dot = dot_product(p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N))
lep_met_cross_x,lep_met_cross_y,lep_met_cross_z = cross_product(p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N))
lep_met_cosine_similarity =cosine_similarity(p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N))

#append features
tX=np.append(tX,np.array([tau_lep_dot,tau_lep_cross_x,tau_lep_cross_y,tau_lep_cross_z,tau_lep_cosine_similarity]).T,axis=1)
tX=np.append(tX,np.array([tau_met_dot,tau_met_cross_x,tau_met_cross_y,tau_met_cross_z,tau_met_cosine_similarity]).T,axis=1)
tX=np.append(tX,np.array([lep_met_dot,lep_met_cross_x,lep_met_cross_y,lep_met_cross_z,lep_met_cosine_similarity]).T,axis=1)

#tau-jet-leading
tau_jet_dot = dot_product(p_tau_x,p_tau_y,p_tau_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
tau_jet_cross_x,tau_jet_cross_y,tau_jet_cross_z = cross_product(p_tau_x,p_tau_y,p_tau_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
tau_jet_cosine_similarity = cosine_similarity(p_tau_x,p_tau_y,p_tau_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)

tX=np.append(tX,np.array([tau_jet_dot,tau_jet_cross_x,tau_jet_cross_y,tau_jet_cross_z,tau_jet_cosine_similarity]).T,axis=1)
#tau-jet-subleading
tau_jet_sub_dot = dot_product(p_tau_x,p_tau_y,p_tau_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
tau_jet_sub_cross_x,tau_jet_sub_cross_y,tau_jet_sub_cross_z = cross_product(p_tau_x,p_tau_y,p_tau_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
tau_jet_sub_cosine_similarity = cosine_similarity(p_tau_x,p_tau_y,p_tau_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([tau_jet_sub_dot,tau_jet_sub_cross_x,tau_jet_sub_cross_y,tau_jet_sub_cross_z,tau_jet_sub_cosine_similarity]).T,axis=1)
#lep-jet-leading
lep_jet_dot = dot_product(p_lep_x,p_lep_y,p_lep_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
lep_jet_cross_x,lep_jet_cross_y,lep_jet_cross_z = cross_product(p_lep_x,p_lep_y,p_lep_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
lep_jet_cosine_similarity = cosine_similarity(p_lep_x,p_lep_y,p_lep_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)

tX=np.append(tX,np.array([lep_jet_dot,lep_jet_cross_x,lep_jet_cross_y,lep_jet_cross_z,lep_jet_cosine_similarity]).T,axis=1)
#lep-jet-subleading
lep_jet_sub_dot = dot_product(p_lep_x,p_lep_y,p_lep_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
lep_jet_sub_cross_x,lep_jet_sub_cross_y,lep_jet_sub_cross_z = cross_product(p_lep_x,p_lep_y,p_lep_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
lep_jet_sub_cosine_similarity = cosine_similarity(p_lep_x,p_lep_y,p_lep_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([lep_jet_sub_dot,lep_jet_sub_cross_x,lep_jet_sub_cross_y,lep_jet_sub_cross_z,lep_jet_sub_cosine_similarity]).T,axis=1)
#met-jet-leading
met_jet_dot = dot_product(p_met_x,p_met_y,np.zeros(N),jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
met_jet_cross_x,met_jet_cross_y,met_jet_cross_z = cross_product(p_met_x,p_met_y,np.zeros(N),jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
met_jet_cosine_similarity = cosine_similarity(p_met_x,p_met_y,np.zeros(N),jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)

tX=np.append(tX,np.array([met_jet_dot,met_jet_cross_x,met_jet_cross_y,met_jet_cross_z,met_jet_cosine_similarity]).T,axis=1)

#met-jet-subleading
met_jet_sub_dot = dot_product(p_met_x,p_met_y,np.zeros(N),jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
met_jet_sub_cross_x,met_jet_sub_cross_y,met_jet_sub_cross_z = cross_product(p_met_x,p_met_y,np.zeros(N),jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
met_jet_sub_cosine_similarity = cosine_similarity(p_met_x,p_met_y,np.zeros(N),jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([met_jet_sub_dot,met_jet_sub_cross_x,met_jet_sub_cross_y,met_jet_sub_cross_z,met_jet_sub_cosine_similarity]).T,axis=1)
#jet-leading-jet-subleading
jet_jet_sub_dot = dot_product(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
jet_jet_sub_cross_x,jet_jet_sub_cross_y,jet_jet_sub_cross_z = cross_product(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
jet_jet_sub_cosine_similarity = cosine_similarity(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([jet_jet_sub_dot,jet_jet_sub_cross_x,jet_jet_sub_cross_y,jet_jet_sub_cross_z,jet_jet_sub_cosine_similarity]).T,axis=1)


In [ ]:
# print(np.count_nonzero(tau_jet_dot==-999))
# print(np.count_nonzero(tau_jet_cross_x==-999))
# print(np.count_nonzero(tau_jet_cosine_similarity==-999))

In [ ]:
#generate determinant vector
N = len(y)
d_vector_0 = determinant_vector(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N))
d_vector_1 = determinant_vector(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z,jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
d_vector_2 = determinant_vector(p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
d_vector_3 = determinant_vector(p_tau_x,p_tau_y,p_tau_z,p_met_x,p_met_y,np.zeros(N),jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
d_vector_4 = determinant_vector(p_tau_x,p_tau_y,p_tau_z,p_met_x,p_met_y,np.zeros(N),jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
d_vector_5 = determinant_vector(p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N),jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
d_vector_6 = determinant_vector(p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y,np.zeros(N),jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
d_vector_7 = determinant_vector(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z,p_tau_x,p_tau_y,p_tau_z)
d_vector_8 = determinant_vector(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z,p_lep_x,p_lep_y,p_lep_z)
d_vector_9 = determinant_vector(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z,jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z,p_met_x,p_met_y,np.zeros(N))

#append features
tX=np.append(tX,np.array([d_vector_0,d_vector_1,d_vector_2,d_vector_3,d_vector_4,d_vector_5]).T,axis=1)
tX=np.append(tX,np.array([d_vector_6,d_vector_7,d_vector_8,d_vector_9]).T,axis=1)

In [ ]:
#sum of px,py,pz
tau_sum = sum_p_xyz(p_tau_x,p_tau_y,p_tau_z)
lep_sum = sum_p_xyz(p_lep_x,p_lep_y,p_lep_z)
met_sum = sum_p_xyz(p_met_x,p_met_y,np.zeros(N))
jet_leading_sum = sum_p_xyz(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
jet_subleading_sum = sum_p_xyz(jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
#append features
tX=np.append(tX,np.array([tau_sum,lep_sum,met_sum,jet_leading_sum,jet_subleading_sum]).T,axis=1)

In [ ]:
#transverse mass
transverse_mass_tau_lep = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_lep_pt,PRI_lep_phi)
transverse_mass_tau_met = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_met,PRI_met_phi)
transverse_mass_tau_jet = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_tau_jet_sub = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
transverse_mass_lep_jet = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_lep_jet_sub = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
#transverse_mass_lep_met = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_met,PRI_met_phi) already have
transverse_mass_met_jet = transverse_mass(PRI_met,PRI_met_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_met_jet_sub= transverse_mass(PRI_met,PRI_met_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
#transverse_mass_jet_jet_sub= transverse_mass(PRI_jet_leading_pt,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi) already have
#append features
tX=np.append(tX,np.array([transverse_mass_tau_lep,transverse_mass_tau_met,transverse_mass_tau_jet,transverse_mass_tau_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([transverse_mass_lep_jet,transverse_mass_lep_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([transverse_mass_met_jet,transverse_mass_met_jet_sub]).T,axis=1)



In [ ]:
#invariant mass
#already have invariant mass between tau and lep and between two jets
invariant_mass_tau_met =invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_met,np.zeros(N),PRI_met_phi)
invariant_mass_lep_met = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,np.zeros(N),PRI_met_phi)
invariant_mass_tau_jet = invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_tau_jet_sub = invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)
invariant_mass_lep_jet = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_lep_jet_sub = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)
invariant_mass_met_jet =invariant_mass(PRI_met,np.zeros(N),PRI_met_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_met_jet_sub = invariant_mass(PRI_met,np.zeros(N),PRI_met_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)



#append features
tX=np.append(tX,np.array([invariant_mass_tau_met,invariant_mass_lep_met]).T,axis=1)
tX=np.append(tX,np.array([invariant_mass_tau_jet,invariant_mass_tau_jet_sub,invariant_mass_lep_jet,invariant_mass_lep_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([invariant_mass_met_jet,invariant_mass_met_jet_sub]).T,axis=1)

In [ ]:
invariant_mass_tau_jet

In [ ]:
tX.shape

#### 1.3 cross-validation function and indice function

In [102]:
def cross_validation(y,tX,k_indices,k,_lambda):
    
    tX_test_indice = k_indices[k]
    tX_train_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tX_train_indice = tX_train_indice.reshape(-1)
    y_test = y[tX_test_indice]
    y_train = y[tX_train_indice]
    tX_test = tX[tX_test_indice]
    tX_train = tX[tX_train_indice]
    
    #split train and test into four groups
    y_train_0,y_train_1,y_train_2,y_train_3,y_test_0,y_test_1,y_test_2,y_test_3,tX_train_0,tX_train_1,tX_train_2,tX_train_3,tX_test_0,tX_test_1,tX_test_2,tX_test_3=split_groups(y_train,y_test,tX_train,tX_test)
    
    
    data_cleaning(tX_train_0)
    data_cleaning(tX_train_1)
    data_cleaning(tX_train_2)
    data_cleaning(tX_train_3)
    
    data_cleaning(tX_test_0)
    data_cleaning(tX_test_1)
    data_cleaning(tX_test_2)
    data_cleaning(tX_test_3)
    
    weight_0 = ridge_regression(y_train_0,tX_train_0,_lambda)
    weight_1 = ridge_regression(y_train_1,tX_train_1,_lambda)
    weight_2 = ridge_regression(y_train_2,tX_train_2,_lambda)
    weight_3 = ridge_regression(y_train_3,tX_train_3,_lambda)
    
    y_pred_0 = predict_labels(weight_0, tX_test_0)
    y_pred_1 = predict_labels(weight_1, tX_test_1)
    y_pred_2 = predict_labels(weight_2, tX_test_2)
    y_pred_3 = predict_labels(weight_3, tX_test_3)
    
    num_0 = np.count_nonzero(y_pred_0==y_test_0)
    num_1 = np.count_nonzero(y_pred_1==y_test_1)
    num_2 = np.count_nonzero(y_pred_2==y_test_2)
    num_3 = np.count_nonzero(y_pred_3==y_test_3)
    
    accuracy_0 = num_0/len(y_pred_0)
    accuracy_1 = num_1/len(y_pred_1)
    accuracy_2 = num_2/len(y_pred_2)
    accuracy_3 = num_3/len(y_pred_3)
    
    num = len(y_test)
    print(f'group 0 accuracy: {accuracy_0}')
    print(f'group 1 accuracy: {accuracy_1}')
    print(f'group 2 accuracy: {accuracy_2}')
    print(f'group 3 accuracy: {accuracy_3}')
    accuracy = (num_0+num_1+num_2+num_3)/num
    
    return accuracy

def split_groups(y_train,y_test,tX_train,tX_test):
    tX_train_jet_num = tX_train[:,22]
    tX_test_jet_num = tX_test[:,22]
    
    y_train_0 = np.copy(y_train[tX_train_jet_num==0])
    y_train_1 = np.copy(y_train[tX_train_jet_num==1])
    y_train_2 = np.copy(y_train[tX_train_jet_num==2])
    y_train_3 = np.copy(y_train[tX_train_jet_num==3])
    
    y_test_0 = np.copy(y_test[tX_test_jet_num==0])
    y_test_1 = np.copy(y_test[tX_test_jet_num==1])
    y_test_2 = np.copy(y_test[tX_test_jet_num==2])
    y_test_3 = np.copy(y_test[tX_test_jet_num==3])
    
    tX_train_0 = np.copy(tX_train[tX_train_jet_num==0])
    tX_train_1 = np.copy(tX_train[tX_train_jet_num==1])
    tX_train_2 = np.copy(tX_train[tX_train_jet_num==2])
    tX_train_3 = np.copy(tX_train[tX_train_jet_num==3])
    
    tX_test_0 = np.copy(tX_test[tX_test_jet_num==0])
    tX_test_1 = np.copy(tX_test[tX_test_jet_num==1])
    tX_test_2 = np.copy(tX_test[tX_test_jet_num==2])
    tX_test_3 = np.copy(tX_test[tX_test_jet_num==3])
    
    return y_train_0,y_train_1,y_train_2,y_train_3,y_test_0,y_test_1,y_test_2,y_test_3,tX_train_0,tX_train_1,tX_train_2,tX_train_3,tX_test_0,tX_test_1,tX_test_2,tX_test_3

def build_k_indices(y,k_fold,seed):
    num_row = y.shape[0]
    interval = int(num_row/k_fold)
    np.random.seed(seed)
    indices=np.random.permutation(num_row)
    k_indices = [indices[k*interval:(k+1)*interval] for k in range (k_fold)]
    return np.array(k_indices)

In [103]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    N = len(y)
    a = tx.T.dot(tx)+lambda_*(2*N)*np.identity(tx.shape[1])
    b = tx.T.dot(y)
    weight = np.linalg.solve(a,b)
    
    return weight

#### 1.5 demo

In [104]:
def demo():
    k_fold = 4
    seed = 3
    k_indices = build_k_indices(y, k_fold, seed)
    total = 0
    
    for k in range(k_fold):
        accuracy = cross_validation(y,tX,k_indices,k,1e-7)
        total  = total+accuracy
        print(f'{k}:{accuracy} ')
    
    average=total/k_fold
    print(f'average accuracy:{average}')

In [105]:
demo()

group 0 accuracy: 0.842745647388433
group 1 accuracy: 0.8038374017376914
group 2 accuracy: 0.8280571563906213
group 3 accuracy: 0.8236574746008708
0:0.826048 
group 0 accuracy: 0.843663718938818
group 1 accuracy: 0.798618219037871
group 2 accuracy: 0.8259862641750519
group 3 accuracy: 0.8228382595924362
1:0.824224 
group 0 accuracy: 0.8410266774454325
group 1 accuracy: 0.7989287185826123
group 2 accuracy: 0.8344188966491407
group 3 accuracy: 0.8264595607927155
2:0.825296 
group 0 accuracy: 0.8381553359526928
group 1 accuracy: 0.7997091211302721
group 2 accuracy: 0.8326504378565116
group 3 accuracy: 0.8242326909350464
3:0.823968 
average accuracy:0.824884
